In [1]:
import torch

from e2cnn import gspaces
from e2cnn import nn

In [2]:
r2_act = gspaces.Rot2dOnR2(N=4)

feat_type_in = nn.FieldType(r2_act, [r2_act.trivial_repr])
feat_type_hid = nn.FieldType(r2_act, 8*[r2_act.regular_repr])
feat_type_out = nn.FieldType(r2_act, 4*[r2_act.regular_repr])

model = nn.SequentialModule(
    nn.R2Conv(feat_type_in, feat_type_hid, kernel_size=3,padding = 1),
    nn.InnerBatchNorm(feat_type_hid),
    nn.ReLU(feat_type_hid),
    nn.R2Conv(feat_type_hid, feat_type_hid, kernel_size=3,padding = 1),
    nn.InnerBatchNorm(feat_type_hid),
    nn.ReLU(feat_type_hid),
    nn.R2Conv(feat_type_hid, feat_type_out, kernel_size=3,padding = 1),
).eval()

/home/jupyter-lsavant/.local/lib/python3.7/site-packages/e2cnn/nn/modules/r2_conv/basisexpansion_singleblock.py:80: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:30.)
  full_mask[mask] = norms.to(torch.uint8)


In [3]:
import h5py

class CustomDataset(torch.utils.data.Dataset):
    #def __init__(self):   

    def __len__(self):
        return 600

    def __getitem__(self, idx):
        with h5py.File('mydataset.hdf5','r') as f:
            outputs = torch.Tensor(f['delta_map'][idx,:,:]).permute([2,0,1])
            inputs = torch.Tensor(f['train'][idx,:,:]).unsqueeze(0)
        return inputs, outputs

In [4]:
train_dataset = CustomDataset()

In [5]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 8)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
for t in range(20):
  cumulative_loss = 0
  for _, (X, Y) in enumerate(train_dataloader):
    X = nn.GeometricTensor(X, feat_type_in)
    y_pred = torch.max(model(X).tensor,1)[0]
    loss = criterion(y_pred, Y[:,0,:,:])
    cumulative_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(t, cumulative_loss)

RuntimeError: Trying to backward through the graph a second time (or directly access saved variables after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved variables after calling backward.

In [ ]:
y_pred.shape

In [ ]:
model(inputs).tensor.shape

In [ ]:
outputs[:,0,:,:].shape